In [0]:
from pyspark.sql.functions import *

In [0]:
expectations={"rule1":"cast_year>1200 and cast_year<2025", "rule2":"show_id is not null"}

In [0]:
import dlt;
@dlt.table(
    name=gold_netflix_cast
)

@dlt.expect_or_drop(expectations)
def netflix_cast():
  data=spark.read.format('delta').load("abfss://silver@pronetflixdata.dfs.core.windows.net/netflix_cast");
  return data;


@dlt.table(
    name=netflix_titles
)


@dlt.expect_or_drop(expectations)
def netflix_titles():
  data2=spark.read.format('delta').load("abfss://silver@pronetflixdata.dfs.core.windows.net/netflix_titles");
  data2.withColumn('effective_date', current_timestamp()) \
    .withColumn('end_date', lit(None)) \
      .withColumn('status', lit(True));
  return data2;

  
@dlt.apply_changes(
  target=gold_netflix_titles,
  source=netflix_titles,
  keys=['show_id'],
  sequence_by='date_added',
  stored_as_scd_type=2

)

  


